In [5]:
import pandas as pd 


In [6]:
data = pd.read_csv("Resources/aviation.csv",encoding='latin1')
data.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.Flight,Air.Carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Report.Status,Publication.Date
0,20200102X82407,Accident,WPR20CA055,12/31/2019,"Elk, CA",United States,39.128611,-123.715833,LLR,Little River,...,Personal,NaN,NaN,NaN,NaN,1.0,VMC,TAKEOFF,Factual,13/01/2020
1,20191231X83852,Accident,CEN20FA049,12/31/2019,"OLATHE, KS",United States,38.846111,-94.736111,OJC,Johnson County Executive,...,Personal,NaN,2.0,NaN,NaN,NaN,VMC,TAKEOFF,Preliminary,8/1/2020
2,20200102X54844,Accident,ANC20CA011,12/31/2019,"Fairbanks, AK",United States,64.666945,-148.133334,NaN,NaN,...,Personal,NaN,NaN,NaN,NaN,2.0,NaN,NaN,Preliminary,2/1/2020
3,20191230X91852,Accident,CEN20CA048,12/30/2019,"GRANBURY, TX",United States,32.365556,-97.645000,NaN,NaN,...,Personal,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Preliminary,31/12/2019
4,20191228X62945,NaN,WPR20CA053,12/28/2019,"Missoula, MT",United States,NaN,NaN,MSO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Preliminary,3/1/2020


In [7]:
aviation_data = data[['Event.Date', 'Country', 'Location', 'Make', 'Model','Amateur.Built', 'Injury.Severity', 'Aircraft.Damage' ,'Latitude', 'Longitude' ]]
aviation_data.head()

,Event.Date,Country,Location,Make,Model,Amateur.Built,Injury.Severity,Aircraft.Damage,Latitude,Longitude
0,12/31/2019,United States,"Elk, CA",Cessna,175,No,Non-Fatal,Substantial,39.128611,-123.715833
1,12/31/2019,United States,"OLATHE, KS",Mooney,M20S,No,Fatal(2),Destroyed,38.846111,-94.736111
2,12/31/2019,United States,"Fairbanks, AK",Cessna,170,No,Non-Fatal,Substantial,64.666945,-148.133334
3,12/30/2019,United States,"GRANBURY, TX",Vans,RV 10,Yes,Non-Fatal,Substantial,32.365556,-97.645000
4,12/28/2019,United States,"Missoula, MT",Cessna,170,No,Unavailable,Substantial,NaN,NaN


In [8]:
us_data = aviation_data[aviation_data['Country']=='United States']
us_data = us_data.dropna()
us_data_pro = us_data[us_data['Amateur.Built'] == 'No']
us_data_clean = us_data_pro.drop('Amateur.Built', 1)
us_data_clean.head()

,Event.Date,Country,Location,Make,Model,Injury.Severity,Aircraft.Damage,Latitude,Longitude
0,12/31/2019,United States,"Elk, CA",Cessna,175,Non-Fatal,Substantial,39.128611,-123.715833
1,12/31/2019,United States,"OLATHE, KS",Mooney,M20S,Fatal(2),Destroyed,38.846111,-94.736111
2,12/31/2019,United States,"Fairbanks, AK",Cessna,170,Non-Fatal,Substantial,64.666945,-148.133334
5,12/28/2019,United States,"LAFAYETTE, LA",Piper,PA 31T,Fatal(5),Destroyed,30.176111,-92.007500
6,12/25/2019,United States,"Headland, AL",Bell,407,Fatal(1),Substantial,31.364167,-85.312500


In [31]:
aviation_renamed = us_data_clean.rename(columns={ 'Event.Date': 'Date', 'Location':'City_State','Injury.Severity': 'Injury_Severity', 'Aircraft.Damage':'Aircraft_Damage'  })
aviation_renamed.head()

,Date,Country,City_State,Make,Model,Injury_Severity,Aircraft_Damage,Latitude,Longitude
0,12/31/2019,United States,"Elk, CA",Cessna,175,Non-Fatal,Substantial,39.128611,-123.715833
1,12/31/2019,United States,"OLATHE, KS",Mooney,M20S,Fatal(2),Destroyed,38.846111,-94.736111
2,12/31/2019,United States,"Fairbanks, AK",Cessna,170,Non-Fatal,Substantial,64.666945,-148.133334
5,12/28/2019,United States,"LAFAYETTE, LA",Piper,PA 31T,Fatal(5),Destroyed,30.176111,-92.007500
6,12/25/2019,United States,"Headland, AL",Bell,407,Fatal(1),Substantial,31.364167,-85.312500


In [32]:
from sqlalchemy import create_engine
import sqlite3



In [39]:
conn = sqlite3.connect('aviation_incidence.sql')
c = conn.cursor()
c.execute('CREATE TABLE AVIATION (Date, Country, City_State, Make, Model, Injury_Severity, Aircraft_Damage,Latitude, Longitude)')
conn.commit()

In [40]:
aviation_renamed.to_sql('AVIATION', conn, if_exists='replace', index = False)

In [41]:
c.execute('''  
SELECT * FROM AVIATION
          ''')

for row in c.fetchall():
    print (row)

', 'Substantial', 33.214166999999996, -96.747777)
('1/8/2001', 'United States', 'Bluefield, WV', 'Cessna', '310R', 'Fatal(1)', 'Destroyed', 37.283889, -81.20611099999999)
('1/8/2001', 'United States', 'WC 368, GM', 'Bell', '206-L3', 'Incident', 'Minor', 28.9025, -93.28277800000001)
('1/8/2001', 'United States', 'Mineral Wells, TX', 'Bell', '206B', 'Fatal(1)', 'Substantial', 32.758055999999996, -98.089166)
('1/7/2001', 'United States', 'Jacksonville, FL', 'Mooney', 'M-20C', 'Non-Fatal', 'Substantial', 30.493610999999998, -81.690833)
('1/7/2001', 'United States', 'Perry, FL', 'Maule', 'MXT-7-180A', 'Non-Fatal', 'Substantial', 30.071111, -83.582223)
('1/7/2001', 'United States', 'Concord, NH', 'Cessna', '172P', 'Non-Fatal', 'Substantial', 43.201667, -71.502222)
('1/6/2001', 'United States', 'Spanish Fork, UT', 'Cessna', '152', 'Non-Fatal', 'Destroyed', 40.184444, -111.703333)
('1/5/2001', 'United States', 'Livermore, CA', 'Cessna', '152', 'Non-Fatal', 'Substantial', 37.683333000000005, -1